In [1]:
import numpy as np
import pandas as pd
import scipy
import random
import time
import os
import matplotlib.pyplot as plt

In [21]:
raw_train_data = pd.read_csv("bank-additional-train.csv",delimiter=",", encoding="utf-8-sig")
raw_train_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,32,management,married,professional.course,unknown,no,no,cellular,jul,mon,...,4,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no
1,41,admin.,married,high.school,no,yes,yes,cellular,apr,mon,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no
2,32,admin.,married,university.degree,no,yes,no,cellular,may,mon,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.299,5099.1,no
3,37,admin.,married,high.school,no,unknown,unknown,cellular,jul,thu,...,3,999,0,nonexistent,1.4,93.918,-42.7,4.968,5228.1,no
4,38,admin.,divorced,university.degree,no,no,no,cellular,jul,tue,...,8,999,0,nonexistent,1.4,93.918,-42.7,4.961,5228.1,no


In [3]:
#See the details in different attributes 
for key in raw_train_data.keys():
    print(raw_train_data[key].value_counts().head(10))

31    1744
32    1663
33    1629
36    1605
35    1570
34    1558
30    1550
37    1335
29    1301
39    1285
Name: age, dtype: int64
admin.           9380
blue-collar      8331
technician       6075
services         3553
management       2646
retired          1536
entrepreneur     1325
self-employed    1276
housemaid         967
unemployed        906
Name: job, dtype: int64
married     22426
single      10410
divorced     4159
unknown        74
Name: marital, dtype: int64
university.degree      10931
high.school             8555
basic.9y                5449
professional.course     4709
basic.4y                3790
basic.6y                2060
unknown                 1559
illiterate                16
Name: education, dtype: int64
no         29301
unknown     7767
yes            1
Name: default, dtype: int64
yes        19447
no         16753
unknown      869
Name: housing, dtype: int64
no         30595
yes         5605
unknown      869
Name: loan, dtype: int64
cellular     23506
telepho

In [22]:
missinglistName = []#For later using
for key in raw_train_data.keys():
    missing=0
    keyword=''
    value = raw_train_data[key].value_counts()
    if 'unknown' in list(value.keys()):
        missing = value['unknown']
        proportion = missing / len(raw_train_data[key])
        missinglistName.append(key)
        print("The number of missing data in %s is %d and the proportion is %f with the keyword unknown" %(key,missing,proportion))

The number of missing data in job is 296 and the proportion is 0.007985 with the keyword unknown
The number of missing data in marital is 74 and the proportion is 0.001996 with the keyword unknown
The number of missing data in education is 1559 and the proportion is 0.042057 with the keyword unknown
The number of missing data in default is 7767 and the proportion is 0.209528 with the keyword unknown
The number of missing data in housing is 869 and the proportion is 0.023443 with the keyword unknown
The number of missing data in loan is 869 and the proportion is 0.023443 with the keyword unknown


In [23]:
missinglistName

['job', 'marital', 'education', 'default', 'housing', 'loan']

In [10]:
#These percentage of missing data in these columns less that 1%, then we will fill them up later

In [5]:
nonexistList=[]
for key in raw_train_data.keys():
    missing=0
    keyword=''
    value = raw_train_data[key].value_counts()
    if 'nonexistent' in list(value.keys()):
        missing = value['nonexistent']
        proportion = missing / len(raw_train_data[key])
        missinglistName.append(key)
        print("The number of missing data in %s is %d and the proportion is %f with the keyword nonexistent" %(key,missing,proportion))

The number of missing data in poutcome is 32018 and the proportion is 0.863741 with the keyword nonexistent


In [24]:
#Beacuse only one attribute has missing data more than 80%, this attribute will only supports less on model. Then we drop this column
raw_train_data = raw_train_data.drop(columns=['poutcome'])
raw_train_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,32,management,married,professional.course,unknown,no,no,cellular,jul,mon,565,4,999,0,1.4,93.918,-42.7,4.962,5228.1,no
1,41,admin.,married,high.school,no,yes,yes,cellular,apr,mon,982,1,999,0,-1.8,93.075,-47.1,1.405,5099.1,no
2,32,admin.,married,university.degree,no,yes,no,cellular,may,mon,188,2,999,0,-1.8,92.893,-46.2,1.299,5099.1,no
3,37,admin.,married,high.school,no,unknown,unknown,cellular,jul,thu,56,3,999,0,1.4,93.918,-42.7,4.968,5228.1,no
4,38,admin.,divorced,university.degree,no,no,no,cellular,jul,tue,419,8,999,0,1.4,93.918,-42.7,4.961,5228.1,no


In [25]:
#Translate non-numerical data to numerical data
def trans_non_numerical_data(df):
    columns = df.columns.values
    for column in columns:
        text_digit_vals ={}
        def convert_to_int(val):
            return text_digit_vals[val]
        
        if df[column].dtype!= np.int64 and df[column].dtype!= np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 1
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    if unique != 'unknown':
                        text_digit_vals[unique] = x
                        x+=1
                    else:
                        text_digit_vals[unique] = 'unknown'
            df[column] = list(map(convert_to_int,df[column]))
            
    return df

In [26]:
raw_train_data = trans_non_numerical_data(raw_train_data)

In [27]:
for key in raw_train_data.keys():
    print(raw_train_data[key].value_counts().head(10))

31    1744
32    1663
33    1629
36    1605
35    1570
34    1558
30    1550
37    1335
29    1301
39    1285
Name: age, dtype: int64
8     9380
1     8331
6     6075
5     3553
10    2646
4     1536
7     1325
9     1276
11     967
3      906
Name: job, dtype: int64
1          22426
3          10410
2           4159
unknown       74
Name: marital, dtype: int64
1          10931
6           8555
4           5449
3           4709
5           3790
2           2060
unknown     1559
7             16
Name: education, dtype: int64
1          29301
unknown     7767
2              1
Name: default, dtype: int64
2          19447
1          16753
unknown      869
Name: housing, dtype: int64
1          30595
2           5605
unknown      869
Name: loan, dtype: int64
1    23506
2    13563
Name: contact, dtype: int64
6     12351
3      6474
1      5576
7      4787
5      3701
9      2379
2       632
8       509
10      499
4       161
Name: month, dtype: int64
1    7757
4    7673
2    7340
5    7269


In [ ]:
# Now we need to deal with the missing data, we use K-Nearest Neighbors (KNN) Imputation.

In [28]:
def dropMissing(dataset,missinglistName):
    #Find the missing line index and put them in to a list
    missingLine = dataset[(dataset['job']=='unknown')|(dataset['marital']=='unknown')|\
                          (dataset['education']=='unknown')|(dataset['default']=='unknown')|\
                          (dataset['housing']=='unknown')|(dataset['loan']=='unknown')]
    missingIndex = list(missingLine.index)
    #Create a new database without three columns with missing data for distance calculation using
    datawithoutMissing = dataset.drop(missinglistName, axis=1)
    return missingIndex,datawithoutMissing   

In [29]:
missingIndex,datawithoutMissing = dropMissing(raw_train_data,missinglistName)

In [30]:
from scipy.spatial import distance_matrix
"""
We calculate  for the missing line with all other complete line in this function and then find the minima distance, 
Copy the value in complete line to missing data line (in the next function) to decreace error. 
"""
#missingIndex is all row number with missing data
def findMinEntropy(data,missingIndex):
    a = data.drop(missingIndex).reset_index()
    completeIndexlist =np.asarray(a.iloc[:,0])
    missingMatrix = np.asarray(data.loc[missingIndex]).astype(np.int64)
    bigMatrix = np.asarray(data.drop(missingIndex)).astype(np.int64) #np.asarray(data).astype(np.int64)
    #matrix = cosine_similarity(missingMatrix,bigMatrix.transpose())
    matrix = scipy.spatial.distance_matrix(missingMatrix,bigMatrix)
    indexList = []
    print("finish calculate the matrix")
    for i in range(matrix.shape[0]):
        index_min = np.argmin(matrix[i,:])
        indexList.append(completeIndexlist[index_min])
    return indexList

In [31]:
completeIndexList = findMinEntropy(datawithoutMissing,missingIndex)

finish calculate the matrix


In [32]:
"""
As last explanation said, we copy the value in completed line for the missing value in missng data line. 
"""
def changeValue(raw_train_data,missingColList,missingIndexList,completeIndexList):
    for i in range(len(missingIndexList)):
        missingLineIndex = missingIndexList[i]
        completeLineIndex = completeIndexList[i]
        for missingCol in missingColList:
            if raw_train_data.at[missingLineIndex,missingCol] == 'unknown':
                value = raw_train_data.at[completeLineIndex, missingCol]#Value is used to fill
                if value == 'unknown':
                    print('This position updata failure. The imformation for the completed Line index is:')
                    print('Line index is: '+ str(completeLineIndex)+ ' Column number is: ' + str(missingCol))
                raw_train_data.at[missingLineIndex, missingCol] = int(value)
    return raw_train_data

In [33]:
#get the clean data
raw_train_data =changeValue(raw_train_data,missinglistName,missingIndex,completeIndexList)

In [34]:
#We save this data for later using whcih means we don't need to do the data Preprocess again at next time.
raw_train_data.to_csv('clean_data.csv',sep=",", index = False, encoding="utf-8-sig")